# Validate Qualitatively

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

from src.config import BASE_LAT_START, BASE_LAT_END, BASE_LON_START, BASE_LON_END
from data_transformer import extract_stations_from_nc
from data_provider import get_station_indices_map

In [ ]:
pred = np.load('predictions/full_pred_plain.npy')
# Convert to hecto pascal
pred[...,1] = pred[...,1] /100
print(pred.shape)

ground_truth = xr.load_dataset("data_sets/ground_truth.nc")
station_indx_map = get_station_indices_map()
gt_stations = extract_stations_from_nc(ground_truth, station_indx_map)  # Is scaled.



DATES = pd.date_range('1807-01-01', freq='D', periods=365).values
DATES = list(map(lambda d: str(d).split('T')[0], DATES))
def date_to_id(date):
    return DATES.index(date)

## Get Average Hottest and Coldest Day according to GT Observations

In [ ]:
temp_df = pd.DataFrame(gt_stations)
temp_df = temp_df[list(filter(lambda c: "_ta" in c,  temp_df.columns))]
temp_df.head(5)

In [ ]:
hottest_id = temp_df.mean(axis=1).argmax()
coolest_id = temp_df.mean(axis=1).argmin()

hottest_date = DATES[hottest_id]
coolest_date = DATES[coolest_id]

print(f"Hottest date: {hottest_date} ({hottest_id})")
print(f"Coolest date: {coolest_date} ({coolest_id})")

## Extract anomalies for those two days

In [ ]:
import statsmodels.api as sm

def extract_ta_cell_anomaly(y_obs, n_doy=365):
    """
    y_obs: one cell over time.
    """
    x_ = np.linspace(0, n_doy - 1, n_doy)  # doy
    x1 = np.sin((2 * np.pi * x_) / n_doy)
    x2 = np.cos((2 * np.pi * x_) / n_doy)
    x3 = np.sin((4 * np.pi * x_) / n_doy)
    x4 = np.cos((4 * np.pi * x_) / n_doy)

    x = pd.DataFrame(data={'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4})
    x_sm = sm.add_constant(x)
    model = sm.OLS(y_obs, x_sm).fit()
    c0, c1, c2, c3, c4 = model.params
    saisonal_component = c0 + c1 * x_sm.x1 + c2 * x_sm.x2 + c3 * x_sm.x3 + c4 * x_sm.x4
    desaisonalized = y_obs - saisonal_component
    # Insert nans again:
   
    return desaisonalized.to_numpy()


# FOR TEMPERATURE
# For each cell, remove seasonality.
ta_anomaly = pred[..., 0].copy()
for row in range(ta_anomaly.shape[1]):
    for column in range(ta_anomaly.shape[2]):
        ta_anomaly[:, row, column] = extract_ta_cell_anomaly(ta_anomaly[:, row, column])



# FOR PRESSURE
# Remove long-term average for each day of the year
inference_ymean = np.load("../data_sets/ymean_sets/inference_ymean.npy")
# Convert to hecto pascal
inference_ymean[...,1] = inference_ymean[...,1] /100
slp_anomaly = pred[..., 1] - inference_ymean[..., 1]

anomaly = np.concatenate([np.expand_dims(ta_anomaly, axis=-1),
                          np.expand_dims(slp_anomaly, axis=-1)], 
                         axis=-1)
print(anomaly.shape)


## Display Climatology and Anomaly Fields

In [ ]:
import cartopy.crs as ccrs

def display_single_field(pred, anomaly, hot_id, cold_id, hot_date, cold_date, savefig=None):
    fig, axes = plt.subplots(2, 2, figsize=(10, 5), subplot_kw={'projection': ccrs.PlateCarree()})
    
    fig.subplots_adjust(right=0.95)
    # Pred
    ta_hot_pred = pred[hot_id, :, :, 0]
    slp_hot_pred = pred[hot_id, :, :, 1]
    ta_cold_pred = pred[cold_id, :, :, 0]
    slp_cold_pred = pred[cold_id, :, :, 1]
    # Anomaly
    ta_hot_anomaly = anomaly[hot_id, :, :, 0]
    slp_hot_anomaly = anomaly[hot_id, :, :, 1]
    ta_cold_anomaly = anomaly[cold_id, :, :, 0]
    slp_cold_anomaly = anomaly[cold_id, :, :, 1]
    
    lon = np.linspace(BASE_LON_START, BASE_LON_END, 64)
    lat = np.linspace(BASE_LAT_START, BASE_LAT_END, 32)
    
    
    vmin_ta_pred = np.min([np.min(ta_hot_pred), np.min(ta_cold_pred)])
    vmax_ta_pred = np.max([np.max(ta_hot_pred), np.max(ta_cold_pred)])
    # print(vmin_ta_pred, vmax_ta_pred)  # -20.1052655768291 31.328528743749473
    levels_pred = np.linspace(-20, 31, 52)
    vmin_ta_anom = np.min([np.min(ta_hot_anomaly), np.min(ta_cold_anomaly)])
    vmax_ta_anom = np.max([np.max(ta_hot_anomaly), np.max(ta_cold_anomaly)])
    print(vmin_ta_anom, vmax_ta_anom)  # -11.774751807514129 6.773982302487481
    levels_anom = np.linspace(-12, 7, 20)
    
    # === ROW 0 -> Climatology ===
    # --- Hot Ta ---
    axes[0,0].set_extent((BASE_LON_START, BASE_LON_END, BASE_LAT_START, BASE_LAT_END), crs=ccrs.PlateCarree())
    axes[0,0].coastlines(resolution='110m', color='gray')
    axes[0,0].set_title("a: " + hot_date, fontsize=11)
    contourf_1 = axes[0,0].contourf(lon, lat, ta_hot_pred, transform=ccrs.PlateCarree(), levels=levels_pred, vmin=vmin_ta_pred, vmax=vmax_ta_pred, cmap='coolwarm')
    #                 tuple (left, bottom, width, height)
    sub_ax_1 = fig.add_axes([0.5, 0.53, 0.02, 0.35])
    fig.colorbar(contourf_1, cax=sub_ax_1, extend='both', boundaries=[vmin_ta_pred, vmax_ta_pred])
    # --- Hot SLP ---
    # Add slp as contour LINES on top
    contourf_1_1 = axes[0,0].contour(lon, lat, slp_hot_pred, transform=ccrs.PlateCarree(), colors='black', levels=7)
    axes[0,0].clabel(contourf_1_1, inline=True, fontsize=7)
    
    # --- Cold Ta ---
    axes[0,1].set_extent((BASE_LON_START, BASE_LON_END, BASE_LAT_START, BASE_LAT_END), crs=ccrs.PlateCarree())
    axes[0,1].coastlines(resolution='110m', color='gray')
    axes[0,1].set_title("b: " + cold_date, fontsize=11)
    contourf_2 = axes[0,1].contourf(lon, lat, ta_cold_pred, transform=ccrs.PlateCarree(), levels=levels_pred, vmin=vmin_ta_pred, vmax=vmax_ta_pred, cmap='coolwarm')
    #                 tuple (left, bottom, width, height)
    sub_ax_2 = fig.add_axes([0.95, 0.53, 0.02, 0.35])
    fig.colorbar(contourf_2, cax=sub_ax_2, extend='both', boundaries=[vmin_ta_pred, vmax_ta_pred])
    # --- Cold SLP ---
    # Add slp as contour LINES on top
    contourf_2_1 = axes[0,1].contour(lon, lat, slp_cold_pred, transform=ccrs.PlateCarree(), colors='black', levels=7)
    axes[0,1].clabel(contourf_2_1, inline=True, fontsize=7)
    
    
    # === ROW 1 -> Anomaly ===
    # --- Hot Ta ---
    axes[1,0].set_extent((BASE_LON_START, BASE_LON_END, BASE_LAT_START, BASE_LAT_END), crs=ccrs.PlateCarree())
    axes[1,0].coastlines(resolution='110m', color='gray')
    # axes[1,0].set_title(hot_date, fontsize=7)
    contourf_3 = axes[1,0].contourf(lon, lat, ta_hot_anomaly, transform=ccrs.PlateCarree(), levels=levels_anom, vmin=vmin_ta_anom, vmax=vmax_ta_anom, cmap='coolwarm')
    #                 tuple (left, bottom, width, height)
    sub_ax_3 = fig.add_axes([0.5, 0.11, 0.02, 0.35])
    fig.colorbar(contourf_3, cax=sub_ax_3)
    # --- Hot SLP ---
    # Add slp as contour LINES on top
    contourf_3_1 = axes[1,0].contour(lon, lat, slp_hot_anomaly, transform=ccrs.PlateCarree(), colors='black', levels=7)
    axes[1,0].clabel(contourf_3_1, inline=True, fontsize=7)
    
    # --- Cold Ta ---
    axes[1,1].set_extent((BASE_LON_START, BASE_LON_END, BASE_LAT_START, BASE_LAT_END), crs=ccrs.PlateCarree())
    axes[1,1].coastlines(resolution='110m', color='gray')
    # axes[1,1].set_title(cold_date, fontsize=7)
    contourf_4 = axes[1,1].contourf(lon, lat, ta_cold_anomaly, transform=ccrs.PlateCarree(), levels=levels_anom, vmin=vmin_ta_anom, vmax=vmax_ta_anom, cmap='coolwarm')
    #                 tuple (left, bottom, width, height)
    sub_ax_4 = fig.add_axes([0.95, 0.11, 0.02, 0.35])
    fig.colorbar(contourf_4, cax=sub_ax_4)
    # --- Cold SLP ---
    # Add slp as contour LINES on top
    contourf_4_1 = axes[1,1].contour(lon, lat, slp_cold_anomaly, transform=ccrs.PlateCarree(), colors='black', levels=7)
    axes[1,1].clabel(contourf_4_1, inline=True, fontsize=7)
    
    fig.text(0.11, 0.56, "Absolute [°C, hPa]", fontsize=11, rotation=90)
    fig.text(0.11, 0.15, "Anomaly [°C, hPa]", fontsize=11, rotation=90)

    
    if savefig:
        plt.savefig("figures/qualitative_single_fields.png", 
            bbox_inches='tight',
            pad_inches=0.1,
            dpi=300,
            )
    
    
display_single_field(pred, anomaly, hottest_id, coolest_id, hottest_date, coolest_date, savefig=True)
    
    

## Display climat, anom and 20c4

In [ ]:
baseline_ta = np.load("mean20CR/ta_20cr_mean.npy")
baseline_slp = np.load("mean20CR/slp_20cr_mean.npy")

In [ ]:
def display_single_field_with_bl(pred, anomaly, bl_ta, bl_slp,
                                 hot_id, cold_id, hot_date, cold_date, savefig=None):
    fig, axes = plt.subplots(3, 2, figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
    
    fig.subplots_adjust(right=0.95)
    # Pred
    ta_hot_pred = pred[hot_id, :, 1:, 0]
    slp_hot_pred = pred[hot_id, :, 1:, 1]
    ta_cold_pred = pred[cold_id, :, 1:, 0]
    slp_cold_pred = pred[cold_id, :, 1:, 1]
    # Anomaly
    ta_hot_anomaly = anomaly[hot_id, :, 1:, 0]
    slp_hot_anomaly = anomaly[hot_id, :, 1:, 1]
    ta_cold_anomaly = anomaly[cold_id, :, 1:, 0]
    slp_cold_anomaly = anomaly[cold_id, :, 1:, 1]
    # Baseline
    ta_hot_bl = bl_ta[hot_id]
    slp_hot_bl = bl_slp[hot_id] / 100
    ta_cold_bl = bl_ta[cold_id]
    slp_cold_bl = bl_slp[cold_id] / 100
    
    lon = np.linspace(BASE_LON_START+1, BASE_LON_END, 63)
    lat = np.linspace(BASE_LAT_START, BASE_LAT_END, 32)
    
    
    vmin_ta_pred = np.min([np.min(ta_hot_pred), np.min(ta_cold_pred)])
    vmax_ta_pred = np.max([np.max(ta_hot_pred), np.max(ta_cold_pred)])
    # print(vmin_ta_pred, vmax_ta_pred)  # -20.1052655768291 31.328528743749473
    levels_pred = np.linspace(-20, 31, 52)
    vmin_ta_anom = np.min([np.min(ta_hot_anomaly), np.min(ta_cold_anomaly)])
    vmax_ta_anom = np.max([np.max(ta_hot_anomaly), np.max(ta_cold_anomaly)])
    print(vmin_ta_anom, vmax_ta_anom)  # -11.774751807514129 6.773982302487481
    levels_anom = np.linspace(-12, 7, 20)
    vmin_ta_bl = np.min([np.min(ta_hot_bl), np.min(ta_cold_bl)])
    vmax_ta_bl = np.max([np.max(ta_hot_bl), np.max(ta_cold_bl)])
    print(vmin_ta_bl, vmax_ta_bl)  # -16.744835 40.652527
    levels_bl = np.linspace(-17, 41, 59)
    
    
    vmin_ta_climat = np.min((vmin_ta_pred, vmin_ta_bl))
    vmax_ta_climat = np.max((vmax_ta_pred, vmax_ta_bl))
    print(vmin_ta_climat, vmax_ta_climat)  # -20.1052655768291 40.65252685546875
    levels_climat = np.linspace(-20, 41, 62)
    
    # === ROW 0 -> Climatology ===
    # --- Hot Ta ---
    axes[0,0].set_extent((BASE_LON_START+1, BASE_LON_END, BASE_LAT_START, BASE_LAT_END), crs=ccrs.PlateCarree())
    axes[0,0].coastlines(resolution='110m', color='gray')
    axes[0,0].set_title("a: " + hot_date, fontsize=11)
    contourf_1 = axes[0,0].contourf(lon, lat, ta_hot_pred, transform=ccrs.PlateCarree(), levels=levels_climat, vmin=vmin_ta_climat, vmax=vmax_ta_climat, cmap='coolwarm')
    #                 tuple (left, bottom, width, height)
    sub_ax_1 = fig.add_axes([0.5, 0.653, 0.02, 0.227])
    fig.colorbar(contourf_1, cax=sub_ax_1, extend='both', boundaries=[vmin_ta_climat, vmax_ta_climat])
    # --- Hot SLP ---
    # Add slp as contour LINES on top
    contourf_1_1 = axes[0,0].contour(lon, lat, slp_hot_pred, transform=ccrs.PlateCarree(), colors='black', levels=7)
    axes[0,0].clabel(contourf_1_1, inline=True, fontsize=7)
    
    # --- Cold Ta ---
    axes[0,1].set_extent((BASE_LON_START+1, BASE_LON_END, BASE_LAT_START, BASE_LAT_END), crs=ccrs.PlateCarree())
    axes[0,1].coastlines(resolution='110m', color='gray')
    axes[0,1].set_title("b: " + cold_date, fontsize=11)
    contourf_2 = axes[0,1].contourf(lon, lat, ta_cold_pred, transform=ccrs.PlateCarree(), levels=levels_climat, vmin=vmin_ta_climat, vmax=vmax_ta_climat, cmap='coolwarm')
    #                 tuple (left, bottom, width, height)
    sub_ax_2 = fig.add_axes([0.95, 0.653, 0.02, 0.227])
    fig.colorbar(contourf_2, cax=sub_ax_2, extend='both', boundaries=[vmin_ta_climat, vmax_ta_climat])
    # --- Cold SLP ---
    # Add slp as contour LINES on top
    contourf_2_1 = axes[0,1].contour(lon, lat, slp_cold_pred, transform=ccrs.PlateCarree(), colors='black', levels=7)
    axes[0,1].clabel(contourf_2_1, inline=True, fontsize=7)
    
    
    # === ROW 1 -> Anomaly ===
    # --- Hot Ta ---
    axes[1,0].set_extent((BASE_LON_START+1, BASE_LON_END, BASE_LAT_START, BASE_LAT_END), crs=ccrs.PlateCarree())
    axes[1,0].coastlines(resolution='110m', color='gray')
    # axes[1,0].set_title(hot_date, fontsize=7)
    contourf_3 = axes[1,0].contourf(lon, lat, ta_hot_anomaly, transform=ccrs.PlateCarree(), levels=levels_anom, vmin=vmin_ta_anom, vmax=vmax_ta_anom, cmap='coolwarm')
    #                 tuple (left, bottom, width, height)
    sub_ax_3 = fig.add_axes([0.5, 0.381, 0.02, 0.228])
    fig.colorbar(contourf_3, cax=sub_ax_3)
    # --- Hot SLP ---
    # Add slp as contour LINES on top
    contourf_3_1 = axes[1,0].contour(lon, lat, slp_hot_anomaly, transform=ccrs.PlateCarree(), colors='black', levels=7)
    axes[1,0].clabel(contourf_3_1, inline=True, fontsize=7)
    
    # --- Cold Ta ---
    axes[1,1].set_extent((BASE_LON_START+1, BASE_LON_END, BASE_LAT_START, BASE_LAT_END), crs=ccrs.PlateCarree())
    axes[1,1].coastlines(resolution='110m', color='gray')
    # axes[1,1].set_title(cold_date, fontsize=7)
    contourf_4 = axes[1,1].contourf(lon, lat, ta_cold_anomaly, transform=ccrs.PlateCarree(), levels=levels_anom, vmin=vmin_ta_anom, vmax=vmax_ta_anom, cmap='coolwarm')
    #                 tuple (left, bottom, width, height)
    sub_ax_4 = fig.add_axes([0.95, 0.381, 0.02, 0.228])
    fig.colorbar(contourf_4, cax=sub_ax_4)
    # --- Cold SLP ---
    # Add slp as contour LINES on top
    contourf_4_1 = axes[1,1].contour(lon, lat, slp_cold_anomaly, transform=ccrs.PlateCarree(), colors='black', levels=7)
    axes[1,1].clabel(contourf_4_1, inline=True, fontsize=7)
    
    
    
    # === ROW 2 -> 20CR / BL ===
    # --- Hot Ta ---
    lon_bl = np.linspace(BASE_LON_START+1.4, BASE_LON_END, 64)
    lat_bl = np.linspace(BASE_LAT_START, BASE_LAT_END, 32)
    axes[2,0].set_extent((BASE_LON_START+1, BASE_LON_END, BASE_LAT_START, BASE_LAT_END), crs=ccrs.PlateCarree())
    axes[2,0].coastlines(resolution='110m', color='gray')
    # axes[1,0].set_title(hot_date, fontsize=7)
    contourf_5 = axes[2,0].contourf(lon_bl, lat_bl, ta_hot_bl, transform=ccrs.PlateCarree(), levels=levels_climat, vmin=vmin_ta_climat, vmax=vmax_ta_climat, cmap='coolwarm')
    #                 tuple (left, bottom, width, height)
    sub_ax_5 = fig.add_axes([0.5, 0.11, 0.02, 0.228])
    fig.colorbar(contourf_5, cax=sub_ax_5, boundaries=[vmin_ta_climat, vmax_ta_climat])
    # --- Hot SLP ---
    # Add slp as contour LINES on top
    contourf_5_1 = axes[2,0].contour(lon_bl, lat_bl, slp_hot_bl, transform=ccrs.PlateCarree(), colors='black', levels=8)
    axes[2,0].clabel(contourf_5_1, inline=True, fontsize=7)
    
    # --- Cold Ta ---
    axes[2,1].set_extent((BASE_LON_START+1, BASE_LON_END, BASE_LAT_START, BASE_LAT_END), crs=ccrs.PlateCarree())
    axes[2,1].coastlines(resolution='110m', color='gray')
    # axes[1,1].set_title(cold_date, fontsize=7)
    contourf_6 = axes[2,1].contourf(lon_bl, lat_bl, ta_cold_bl, transform=ccrs.PlateCarree(), levels=levels_climat, vmin=vmin_ta_climat, vmax=vmax_ta_climat, cmap='coolwarm')
    #                 tuple (left, bottom, width, height)
    sub_ax_6 = fig.add_axes([0.95, 0.11, 0.02, 0.228])
    fig.colorbar(contourf_6, cax=sub_ax_6, boundaries=[vmin_ta_climat, vmax_ta_climat])
    # --- Cold SLP ---
    # Add slp as contour LINES on top
    contourf_6_1 = axes[2,1].contour(lon_bl, lat_bl, slp_cold_bl, transform=ccrs.PlateCarree(), colors='black', levels=8)
    axes[2,1].clabel(contourf_6_1, inline=True, fontsize=7)
    
    
    fig.text(0.11, 0.68, "Absolute [°C, hPa]", fontsize=11, rotation=90)
    fig.text(0.11, 0.41, "Anomaly [°C, hPa]", fontsize=11, rotation=90)
    fig.text(0.11, 0.16, "20CR [°C, hPa]", fontsize=11, rotation=90)

    
    if savefig:
        plt.savefig("figures/qualitative_single_fields_bl.png", 
            bbox_inches='tight',
            pad_inches=0.1,
            dpi=300,
            )
    
    
display_single_field_with_bl(pred, anomaly, baseline_ta, baseline_slp, hottest_id, coolest_id, hottest_date, coolest_date, savefig=True)